# Segmenting and Clustering Neighborhoods in Toronto

## Step 1: Scraping and Structuring the Data

In [38]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize

In [47]:
# since Wikipedia is cencored in my country, I had to use the wikizero link for the same information.
# I manually copied the page source to file postal_codes.html and parsed from there.
#url_for_postal_codes = 'https://en.0wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#response = requests.get(url_for_postal_codes)

with open('postal_codes.html') as f:
    page_content = f.read()

In [49]:
from collections import OrderedDict

soup = BeautifulSoup(page_content, 'lxml')

# get the part where the table is defined
table = soup.find('table')

# read header
header = table.find('tr').text.strip().split("\n")

# create empty dataframe
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

processed_data = OrderedDict()

# visit all table rows excluding the header
for row in table.find_all('tr')[1:]:
    code, borough, neigh = row.text.strip().split("\n")
    
    if borough != 'Not assigned':
        if neigh == 'Not assigned':
            neigh = borough
    
        if code in processed_data:
            processed_data[code][1] += ", %s" % neigh
        else:
            processed_data.update({code: [borough, neigh]})
        
for pcode, value in processed_data.items():
    df = df.append({'PostalCode': pcode, 'Borough': value[0], 'Neighborhood': value[1]}, ignore_index=True)

In [51]:
df.shape

(103, 3)